In [ ]:
!pip install kaggle

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import matplotlib

from tensorflow.keras.utils import to_categorical
from google.colab import files
from keras import layers
from keras.models import Sequential, load_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

%config InlineBackend.figure_format = 'retina'

In [ ]:
def load_data(train_path, test_path):
  with open(train_path, mode='rb') as f:
    train = pickle.load(f)
  
  with open(test_path, mode='rb') as f:
    test = pickle.load(f)
      
  print("Data loaded")

  return train, test

In [ ]:
training_file = "/content/drive/MyDrive/Colab Notebooks/DicodingMLTerapan/Datasets/GermanTrafficSign/train.p"
testing_file = "/content/drive/MyDrive/Colab Notebooks/DicodingMLTerapan/Datasets/GermanTrafficSign/test.p" 

train, test = load_data(training_file, testing_file)

In [ ]:
signs_name_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DicodingMLTerapan/Datasets/GermanTrafficSign/signnames.csv')
signs_name_df.head()

In [ ]:
sign_name = signs_name_df.SignName.values
sign_name

In [ ]:
X, y = train['features'], train['labels']

data = np.array(X)
labels = np.array(y)

print(data.shape, labels.shape)

X_test, y_test = test['features'], test['labels']

X_test = np.array(X_test)
y_test = np.array(y_test)

print(X_test.shape, y_test.shape)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    data, labels, test_size=.1, random_state=0)

print(X_train.shape, X_val.shape, X_test.shape)

In [ ]:
n_labels = np.unique(y_train).size

def hist_data(y_data, title=None, ax=None, **kwargs):
    if not ax :
        fig = plt.figure()
        ax = fig.add_subplot(111)
    ax.hist(y_data, np.arange(-0.5, n_labels+1.5), stacked=True, **kwargs)
    ax.set_xlim(-0.5, n_labels-0.5)
    if 'label' in kwargs : ax.legend()
    if title : ax.set_title(title)
        
fig, ax = plt.subplots(1,3, figsize=(18,5))
hist_data(y_train, title='Distribusi kelas pada data training', ax=ax[0])
hist_data(y_val, title='Distribusi kelas pada data validasi', ax=ax[1], color='black')
hist_data(y_test, title='Distribusi kelas pada data test', ax=ax[2], color='grey')

In [ ]:
y_train = to_categorical(y_train, n_labels)
y_val = to_categorical(y_val, n_labels)

In [ ]:
class MyCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs['acc'] >= 0.96):
      print('\nAkurasi telah mencapai 96%, Stop training!')
      self.model.stop_training = True

callback = MyCallback()

In [ ]:
def build_model(n_classes):
  model = Sequential()
  # CNN layer 1
  model.add(layers.Conv2D(32, (5, 5), activation='relu', input_shape=X_train.shape[1:]))
  model.add(layers.Conv2D(32, (5, 5), activation='relu'))
  model.add(layers.MaxPooling2D(2, 2))
  model.add(layers.Dropout(0.25))
  # CNN layer 2
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D(2, 2))
  model.add(layers.Dropout(0.25))
  ## Fully Connected layer
  model.add(layers.Flatten())
  model.add(layers.Dense(256, activation='relu'))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(n_classes, activation='softmax'))

  model.summary()

  return model

In [ ]:
epoch = 25

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.CategoricalCrossentropy()

model = build_model(n_labels)

model.compile(optimizer=optimizer,
              loss=loss,
              metrics=['acc'])

In [ ]:
hist = model.fit(
    X_train,
    y_train,
    batch_size=32,
    epochs=epoch,
    validation_data=(X_val, y_val),
    callbacks=[callback],
    verbose=1
  )

In [ ]:
# Plot graph for accuracy
plt.plot(hist.history['acc'], label='Training Accuracy')
plt.plot(hist.history['val_acc'], label='Val Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot graph for loss
plt.plot(hist.history['loss'], label='Training Loss')
plt.plot(hist.history['val_loss'], label='Val Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
pred = np.argmax(model.predict(X_test), axis=-1)
accuracy_score(y_test, pred)

In [ ]:
print(classification_report(y_test, pred))